In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Earthquake\ Shelter\ Project

Mounted at /content/drive
/content/drive/MyDrive/Earthquake Shelter Project


In [28]:
import community.community_louvain as community_louvain
import pandas as pd
import pickle
import folium
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

![](https://drive.google.com/uc?export=view&id=16jviwtyIvY15ifcAKpUVXKneqKAk1okc)



In [3]:
def read_pickle(dataset_name):
  with open('data/' + dataset_name + '.pkl', 'rb') as file_:
    return pickle.load(file_)

In [4]:
features = read_pickle('all_features')
features

,mesh_id,latitude,longitude,area,rice_paddy,other_agriculture,forest,wasteland,building,road_transport,...,dist_to_water,admin_code,2015,total,0-14,15-64,65+,75+,80+,shelter_count
0,53392307,35.50000,139.4625,1047954,0.00,0.00,0.05,0.00,0.739999,0.05,...,0.000000,13209,8133.3263,8428.9629,1125.1046,5516.7014,1787.1569,972.5568,536.7875,1
1,53392308,35.50000,139.4750,1047946,0.00,0.15,0.04,0.00,0.680000,0.11,...,1133.933309,13209,3263.0956,3392.0479,416.3255,2249.2543,726.4682,392.1746,239.3943,0
2,53392317,35.50833,139.4625,1047847,0.00,0.00,0.00,0.00,0.940001,0.03,...,0.000000,13209,9147.2431,9151.1161,1192.9315,6033.2625,1924.9220,1212.4851,808.1996,2
3,53392326,35.51667,139.4500,1047748,0.00,0.02,0.00,0.00,0.880001,0.01,...,0.000000,13209,8092.2569,8307.2410,1168.1542,5368.2292,1770.8576,1078.1972,652.0640,0
4,53392327,35.51667,139.4625,1047740,0.00,0.05,0.00,0.00,0.880001,0.01,...,925.174449,13209,12250.3348,12274.6534,1340.3404,7578.0816,3356.2314,2048.7194,1272.4508,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,53396107,35.83333,139.2125,1043833,0.00,0.05,0.86,0.08,0.010000,0.00,...,1459.941257,13205,21.0371,15.2681,1.4179,4.0062,9.8440,6.1216,5.1849,0
1374,53396108,35.83333,139.2250,1043824,0.00,0.03,0.91,0.04,0.020000,0.00,...,1849.483501,13205,2.0309,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0
1375,53396112,35.84167,139.1500,1043773,0.00,0.00,1.00,0.00,0.000000,0.00,...,2773.751808,13308,16.9579,13.9130,0.7987,8.0011,5.1132,3.2897,2.3840,0
1376,53396201,35.83333,139.2625,1043796,0.03,0.14,0.63,0.00,0.030000,0.00,...,1459.296689,13205,194.1528,168.2823,7.3651,75.4651,85.4520,57.0754,35.6905,0


In [41]:
bottom_lat = 35.5
bottom_lon = 139.4625
lat_width = 75 / 9000
lon_height = 0.0125
error = 0.001

In [75]:
colors = ['#264653', '#2A9D8F', '#E9C46A', '#F4A261', '#E76F51', '#581845', '#C70039', '#FF5733', '#FFC300', '#DAF7A6',
          '#457B9D', '#9E2A2B', '#FF5733', '#FFC300', '#7209B7', '#3A0CA3', '#F72585', '#FFBE0B', '#3F681C', '#8EBA42']

In [76]:
def create_map(features, community):
  m2 = folium.Map(location=[35.7015082, 139.5221197], zoom_start=10.3, tiles='cartodbpositron') # Higashi-Koganei station

  for coord in zip(features['latitude'], features['longitude'], features[community]):
    bottom_left = (coord[0], coord[1])
    top_right = (coord[0] + lat_width, coord[1] + lon_height)

    folium.Rectangle(bounds=[bottom_left, top_right], color=colors[coord[2]],
                     fill=True, fill_color=colors[coord[2]]).add_to(m2)

  return m2

## Community 1: No node attributes (control)

In [16]:
G = nx.Graph()

In [17]:
for i, row in features.iterrows():
  G.add_node(i, lat=row['latitude'], lon=row['longitude'], **row.drop(['latitude', 'longitude']))

In [18]:
for i, row1 in features.iterrows():
  for j, row2 in features.iterrows():
    lat_cond = abs(abs(row1['latitude'] - row2['latitude']) - lat_width) < error
    lon_cond = abs(abs(row1['longitude'] - row2['longitude']) - lon_height) < error
    if lat_cond or lon_cond:
       G.add_edge(i, j)

In [26]:
partition1 = community_louvain.best_partition(G)
features['community1'] = pd.Series(partition1)

In [68]:
m_p1 = create_map(features, 'community1')
m_p1

## Community 2: Louvain's (resolution=1.0) with Cosine Similarity

In [38]:
features_only = features.drop(['mesh_id', 'latitude', 'longitude', 'area', 'admin_code',
                               '2015', 'total', 'community1', 'community2'], axis=1)
features_only

,rice_paddy,other_agriculture,forest,wasteland,building,road_transport,rail_transport,other_site,river_wetland,seashore,sea,golf_course,unknown,dist_to_water,0-14,15-64,65+,75+,80+,shelter_count
0,0.00,0.00,0.05,0.00,0.739999,0.05,0.04,0.07,0.05,0.0,0.0,0.00,0.0,0.000000,1125.1046,5516.7014,1787.1569,972.5568,536.7875,1
1,0.00,0.15,0.04,0.00,0.680000,0.11,0.00,0.02,0.00,0.0,0.0,0.00,0.0,1133.933309,416.3255,2249.2543,726.4682,392.1746,239.3943,0
2,0.00,0.00,0.00,0.00,0.940001,0.03,0.00,0.00,0.03,0.0,0.0,0.00,0.0,0.000000,1192.9315,6033.2625,1924.9220,1212.4851,808.1996,2
3,0.00,0.02,0.00,0.00,0.880001,0.01,0.00,0.07,0.02,0.0,0.0,0.00,0.0,0.000000,1168.1542,5368.2292,1770.8576,1078.1972,652.0640,0
4,0.00,0.05,0.00,0.00,0.880001,0.01,0.00,0.06,0.00,0.0,0.0,0.00,0.0,925.174449,1340.3404,7578.0816,3356.2314,2048.7194,1272.4508,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,0.00,0.05,0.86,0.08,0.010000,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,1459.941257,1.4179,4.0062,9.8440,6.1216,5.1849,0
1374,0.00,0.03,0.91,0.04,0.020000,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,1849.483501,0.0000,0.0000,0.0000,0.0000,0.0000,0
1375,0.00,0.00,1.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,2773.751808,0.7987,8.0011,5.1132,3.2897,2.3840,0
1376,0.03,0.14,0.63,0.00,0.030000,0.00,0.00,0.04,0.00,0.0,0.0,0.13,0.0,1459.296689,7.3651,75.4651,85.4520,57.0754,35.6905,0


In [39]:
similarity_matrix = cosine_similarity(features_only)

In [36]:
G2 = nx.from_numpy_array(similarity_matrix)

In [70]:
partition2 = community_louvain.best_partition(G2)
features['community2'] = pd.Series(partition2)

In [71]:
m_p2 = create_map(features, 'community2')
m_p2

## Community 3: Louvain's (resolution=2.0) with Cosine Similarity

In [87]:
partition3 = community_louvain.best_partition(G2, resolution=1.5)
features['community3'] = pd.Series(partition3)

In [88]:
set(partition3.values())

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [89]:
m_p3 = create_map(features, 'community3')
m_p3

IndexError: list index out of range